In [19]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
key = '1tZQKilfR2iWUCopX3do8g'

In [2]:
# 형식 지정 출력
formatted_now = datetime.now().strftime("%Y-%m-%d %H:%M")
print("형식 지정 날짜와 시간:", formatted_now)

형식 지정 날짜와 시간: 2025-01-21 12:47


In [3]:
formatted_now == '2025-01-20 03:48'

False

In [46]:
start_day = (datetime.now() - timedelta(days = 1)).strftime("%Y%m%d%H00")
print(start_day)

202501201400


In [47]:
end_day = datetime.now().strftime("%Y%m%d%H00")
print(end_day)

202501211400


In [48]:
# https://apihub.kma.go.kr/
# https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?stn=108&help=1&authKey=tfCSyWZ-QEewkslmflBHbg
# https://www.weather.go.kr/w/weather/forecast/short-term.do 단기예보
column_names = [
    "TM", "STN", "WD", "WS", "GST_WD", "GST_WS", "GST_TM", "PA", "PS", "PT", 
    "PR", "TA", "TD", "HM", "PV", "RN", "RN_DAY", "RN_JUN", "RN_INT", 
    "SD_HR3", "SD_DAY", "SD_TOT", "WC", "WP", "WW", "CA_TOT", "CA_MID", 
    "CH_MIN", "CT", "CT_TOP", "CT_MID", "CT_LOW", "VS", "SS", "SI", "ST_GD", 
    "TS", "TE_005", "TE_01", "TE_02", "TE_03", "ST_SEA", "WH", "BF", "IR", "IX"
]

predict_data = pd.read_csv(f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?tm1={start_day}&tm2={end_day}&stn=108&help=1&authKey=1tZQKilfR2iWUCopX3do8g', encoding='cp949', comment='#', sep=r'\s+', header=None, names=column_names)

In [49]:
predict_data

,TM,STN,WD,WS,GST_WD,GST_WS,GST_TM,PA,PS,PT,...,TS,TE_005,TE_01,TE_02,TE_03,ST_SEA,WH,BF,IR,IX
0,202501201400,108,20,2.1,-9,-9.0,-9,1004.1,1014.6,-9,...,7.0,-1.3,-0.6,-0.6,0.0,-9,-9.0,-9,3,1
1,202501201500,108,18,1.9,-9,-9.0,-9,1004.0,1014.5,7,...,5.6,-1.2,-0.6,-0.6,0.0,-9,-9.0,-9,3,1
2,202501201600,108,23,1.6,-9,-9.0,-9,1004.5,1015.0,-9,...,3.7,-1.1,-0.6,-0.6,0.1,-9,-9.0,-9,3,1
3,202501201700,108,23,2.2,-9,-9.0,-9,1005.0,1015.5,-9,...,0.5,-1.0,-0.6,-0.6,0.0,-9,-9.0,-9,3,1
4,202501201800,108,27,1.9,-9,-9.0,-9,1005.8,1016.4,2,...,-0.4,-0.9,-0.6,-0.7,0.0,-9,-9.0,-9,3,1
5,202501201900,108,27,1.3,-9,-9.0,-9,1006.5,1017.1,-9,...,-0.4,-1.0,-0.6,-0.6,0.0,-9,-9.0,-9,3,2
6,202501202000,108,27,1.4,-9,-9.0,-9,1007.0,1017.7,-9,...,-0.5,-0.9,-0.6,-0.6,0.1,-9,-9.0,-9,3,2
7,202501202100,108,32,1.5,-9,-9.0,-9,1007.6,1018.3,2,...,-0.6,-0.9,-0.6,-0.7,0.1,-9,-9.0,-9,3,1
8,202501202200,108,27,1.7,-9,-9.0,-9,1008.1,1018.8,-9,...,-0.7,-0.9,-0.6,-0.6,0.0,-9,-9.0,-9,3,2
9,202501202300,108,27,1.3,-9,-9.0,-9,1008.3,1019.1,-9,...,-0.8,-1.0,-0.6,-0.7,0.0,-9,-9.0,-9,3,2


In [54]:
len(column_names)

46

In [55]:
#사용할 컬럼 ['tm','hm','pv','td','pa','m005Te','m01Te','m02Te','m03Te','clfmAbbrCd','lcsCh']
predict_data = predict_data[['TM', 'HM', 'PV', 'TD', 'PA', 'TE_005', 'TE_01', 'TE_02', 'TE_03', 'CT', 'CH_MIN']]

In [56]:
predict_data

,TM,HM,PV,TD,PA,TE_005,TE_01,TE_02,TE_03,CT,CH_MIN
0,202501201400,58.0,5.8,-0.7,1004.1,-1.3,-0.6,-0.6,0.0,Sc,9
1,202501201500,55.0,5.6,-1.1,1004.0,-1.2,-0.6,-0.6,0.0,Sc,9
2,202501201600,52.0,5.3,-2.0,1004.5,-1.1,-0.6,-0.6,0.1,Sc,10
3,202501201700,54.0,5.1,-2.5,1005.0,-1.0,-0.6,-0.6,0.0,Sc,12
4,202501201800,61.0,5.3,-2.0,1005.8,-0.9,-0.6,-0.7,0.0,-,-9
5,202501201900,66.0,5.4,-1.8,1006.5,-1.0,-0.6,-0.6,0.0,-,-9
6,202501202000,70.0,5.3,-2.0,1007.0,-0.9,-0.6,-0.6,0.1,-,-9
7,202501202100,74.0,5.5,-1.5,1007.6,-0.9,-0.6,-0.7,0.1,-,-9
8,202501202200,76.0,5.4,-1.8,1008.1,-0.9,-0.6,-0.6,0.0,-,-9
9,202501202300,79.0,5.4,-1.8,1008.3,-1.0,-0.6,-0.7,0.0,-,-9


In [57]:
predict_data.loc[:,'CT'] = predict_data['CT'].map(lambda x: 'no_clouds' if x.strip() == '-' else x)

/var/folders/kk/gw33n2fd53b2m5g5p87946br0000gn/T/ipykernel_27282/1534879559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data.loc[:,'CT'] = predict_data['CT'].map(lambda x: 'no_clouds' if x.strip() == '-' else x)


In [58]:
#predict_data.loc[:,'CT']
def no_clouds_chmin(row):
    if row['CT'] == 'no_clouds':
        row['CH_MIN'] = 0.0
    return row
predict_data.loc[:,['CT','CH_MIN']] = predict_data[['CT','CH_MIN']].apply(lambda x: no_clouds_chmin(x), axis=1)

/var/folders/kk/gw33n2fd53b2m5g5p87946br0000gn/T/ipykernel_27282/2455635915.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data.loc[:,['CT','CH_MIN']] = predict_data[['CT','CH_MIN']].apply(lambda x: no_clouds_chmin(x), axis=1)
/var/folders/kk/gw33n2fd53b2m5g5p87946br0000gn/T/ipykernel_27282/2455635915.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  predict_data.loc[:,['CT','CH_MIN']] = predict_data[['CT','CH_MIN']].apply(lambda x: no_clouds_chmin(x), axis=1)


In [59]:
predict_data

,TM,HM,PV,TD,PA,TE_005,TE_01,TE_02,TE_03,CT,CH_MIN
0,202501201400,58.0,5.8,-0.7,1004.1,-1.3,-0.6,-0.6,0.0,Sc,9.0
1,202501201500,55.0,5.6,-1.1,1004.0,-1.2,-0.6,-0.6,0.0,Sc,9.0
2,202501201600,52.0,5.3,-2.0,1004.5,-1.1,-0.6,-0.6,0.1,Sc,10.0
3,202501201700,54.0,5.1,-2.5,1005.0,-1.0,-0.6,-0.6,0.0,Sc,12.0
4,202501201800,61.0,5.3,-2.0,1005.8,-0.9,-0.6,-0.7,0.0,no_clouds,0.0
5,202501201900,66.0,5.4,-1.8,1006.5,-1.0,-0.6,-0.6,0.0,no_clouds,0.0
6,202501202000,70.0,5.3,-2.0,1007.0,-0.9,-0.6,-0.6,0.1,no_clouds,0.0
7,202501202100,74.0,5.5,-1.5,1007.6,-0.9,-0.6,-0.7,0.1,no_clouds,0.0
8,202501202200,76.0,5.4,-1.8,1008.1,-0.9,-0.6,-0.6,0.0,no_clouds,0.0
9,202501202300,79.0,5.4,-1.8,1008.3,-1.0,-0.6,-0.7,0.0,no_clouds,0.0


In [60]:
label_encoding_dict = {
    "no_clouds": 0,
    "Sc": 1,
    "Ci": 2,
    "ScAs": 3,
    "StNs": 4,
    "Ac": 5,
    "As": 6,
    "ScAc": 7,
    "ScCi": 8,
    "CuSc": 9,
    "AcCi": 10,
    "StAs": 11,
    "StSc": 12,
    "St": 13,
    "Cu": 14,
    "Cs": 15,
    "CuCi": 16,
    "ScNs": 17,
    "AsCi": 18,
    "CuAc": 19,
    "StCi": 20,
    "AcCs": 21,
    "ScCs": 22,
    "Cc": 23,
    "ScAcCi": 24,
    "CuAs": 25,
    "CsCi": 26,
    "AsAc": 27,
    "CbSt": 28,
    "CuSt": 29,
    "CbStNs": 30,
    "AcCc": 31,
    "AsCs": 32,
    "CbSc": 33,
    "CbCu": 34,
    "CbNs": 35,
    "Ns": 36,
    "ScCc": 37,
    "CuScCi": 38,
    "StAc": 39,
    "CcCi": 40,
    "CuScAs": 41,
    "CbAs": 42,
    "CuScAc": 43,
    "ScAcCs": 44,
    "Cb": 45,
    "CuScAcCi": 46,
    "CuScCc": 47,
    "CuCs": 48,
    "CbCi": 49,
    "CuAcCi": 50,
    "CuCc": 51,
    "ScAcCc": 52,
    "CuScCs": 53,
    "AsCc": 54,
    "ScCcCi": 55,
    "CbAc": 56,
    "CbCuNs": 57,
    "CuNs": 58,
    "CuScAsCi": 59,
    "ScAsCi": 60,
    "AcCcCi": 61,
    "CuScNs": 62,
    "AsSc": 63,
    "ScCu": 64,
    "CuScAcCc": 65,
    "CbScNs": 66,
    "CuScCcCi": 67,
    "CbScCs": 68,
    "CuCcCi": 69,
    "CuAcCc": 70,
    "StAsCi": 71,
    "CbScAs": 72,
    "ScAsCs": 73
}

In [61]:
predict_data['CT'] = predict_data['CT'].map(lambda x : label_encoding_dict[x])

/var/folders/kk/gw33n2fd53b2m5g5p87946br0000gn/T/ipykernel_27282/552236467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['CT'] = predict_data['CT'].map(lambda x : label_encoding_dict[x])


In [62]:
predict_data.columns

Index(['TM', 'HM', 'PV', 'TD', 'PA', 'TE_005', 'TE_01', 'TE_02', 'TE_03', 'CT',
       'CH_MIN'],
      dtype='object')

In [63]:
scaler = MinMaxScaler()
lstm_predict_X = scaler.fit_transform(predict_data.loc[1:,['HM','TD','PA','TE_005','TE_01','TE_02','TE_03','CT','CH_MIN']]).reshape(-1,9)

In [64]:
lstm_predict_X = predict_X.reshape(-1,24,9)

In [65]:
lstm_load_model_3h = load_model('/Volumes/ESD-ISO/project_preson/LSTM/LSTM_models/3h_models/3h_epoch-93-acc-0.8595-val_acc-0.7893.h5')
lstm_load_model_6h = load_model('/Volumes/ESD-ISO/project_preson/LSTM/LSTM_models/6h_models/6h_epoch-68-acc-0.8379-val_acc-0.7893.h5')
lstm_load_model_12h = load_model('/Volumes/ESD-ISO/project_preson/LSTM/LSTM_models/12h_models/12h_epoch-93-acc-0.8156-val_acc-0.7741.h5')
lstm_load_model_24h = load_model('/Volumes/ESD-ISO/project_preson/LSTM/LSTM_models/24h_models/24h_epoch-100-acc-0.7819-val_acc-0.7005.h5')

In [66]:
lstm_load_model_3h.predict(lstm_predict_X)

2025-01-21 14:15:50.936195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:51.003913: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:51.011151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:51.124419: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 574ms/step


array([[0.7553115]], dtype=float32)

In [67]:
lstm_load_model_6h.predict(lstm_predict_X)

2025-01-21 14:15:51.533016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:51.594723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:51.602012: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:51.716575: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 577ms/step


array([[0.9992656]], dtype=float32)

In [68]:
lstm_load_model_12h.predict(lstm_predict_X)

2025-01-21 14:15:52.129795: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:52.198456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:52.205584: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:52.322281: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 580ms/step


array([[0.99995065]], dtype=float32)

In [69]:
lstm_load_model_24h.predict(lstm_predict_X)

2025-01-21 14:15:54.742326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:54.803061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:54.812070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-21 14:15:54.922181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 800ms/step


array([[0.98558444]], dtype=float32)

In [52]:
# https://apihub.kma.go.kr/
# https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?stn=108&help=1&authKey=tfCSyWZ-QEewkslmflBHbg
# https://www.weather.go.kr/w/weather/forecast/short-term.do 단기예보
column_names = [
    "TM", "STN", "WD", "WS", "GST_WD", "GST_WS", "GST_TM", "PA", "PS", "PT", 
    "PR", "TA", "TD", "HM", "PV", "RN", "RN_DAY", "RN_JUN", "RN_INT", 
    "SD_HR3", "SD_DAY", "SD_TOT", "WC", "WP", "WW", "CA_TOT", "CA_MID", 
    "CH_MIN", "CT", "CT_TOP", "CT_MID", "CT_LOW", "VS", "SS", "SI", "ST_GD", 
    "TS", "TE_005", "TE_01", "TE_02", "TE_03", "ST_SEA", "WH", "BF", "IR", "IX"
]

predict_data = pd.read_csv(f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?tm1={start_day}&tm2={end_day}&stn=108&help=1&authKey=1tZQKilfR2iWUCopX3do8g', encoding='cp949', comment='#', sep=r'\s+', header=None, names=column_names)

In [73]:
DNN_predict_data = predict_data.loc[:,['HM','PV','TD','PA','TE_005','TE_01','TE_02','TE_03','CT','CH_MIN']]

In [77]:
dnn_predict_X = DNN_predict_data.iloc[-1]

In [81]:
scaler = MinMaxScaler()
dnn_predict_X = scaler.fit_transform(np.array(dnn_predict_X).reshape(-1,1)).reshape(-1,10)

In [43]:
dnn_load_model_3h = load_model('/Volumes/ESD-ISO/project_preson/DNN/DNN_models/3h_models/3h_epoch-97-acc-0.7390-val_acc-0.7369.h5')
dnn_load_model_6h = load_model('/Volumes/ESD-ISO/project_preson/DNN/DNN_models/6h_models/6h_epoch-98-acc-0.6667-val_acc-0.6644.h5')
dnn_load_model_12h = load_model('/Volumes/ESD-ISO/project_preson/DNN/DNN_models/12h_models/12h_epoch-78-acc-0.6100-val_acc-0.6193.h5')
dnn_load_model_24h = load_model('/Volumes/ESD-ISO/project_preson/DNN/DNN_models/24h_models/24h_epoch-22-acc-0.5840-val_acc-0.6015.h5')

In [82]:
dnn_load_model_3h.predict(dnn_predict_X)

2025-01-21 14:33:53.188932: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 598ms/step


array([[1.4952436e-05]], dtype=float32)

In [83]:
dnn_load_model_6h.predict(dnn_predict_X)

1/1 [==============================] - 0s 73ms/step


2025-01-21 14:34:02.489617: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


array([[0.33505097]], dtype=float32)

In [84]:
dnn_load_model_12h.predict(dnn_predict_X)

1/1 [==============================] - 0s 69ms/step


2025-01-21 14:34:06.728213: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


array([[0.36636627]], dtype=float32)

In [85]:
dnn_load_model_24h.predict(dnn_predict_X)

1/1 [==============================] - 0s 68ms/step


2025-01-21 14:34:12.788435: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


array([[0.3317835]], dtype=float32)